# 내 문서 기반 RAG 챗봇 완성하기

* 지금까지의 실습 결과를 종합하여,
* 하나의 완성형 RAG 챗봇을 구성합니다.
* 사용자 질문에 대해, 내 문서로부터 유사한 내용을 검색하고,
* LLM이 검색 결과를 바탕으로 응답을 생성하도록 합니다.

### 1. 환경 로드 및 임베딩 모델 세팅

In [8]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
import os

load_dotenv()

# 임베딩 모델 (E5)
embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")

# 벡터 인덱스 불러오기
faiss_index = FAISS.load_local(
    "embeddings/faiss_index_e5",
    embeddings=embedding,
    allow_dangerous_deserialization=True
)
retriever = faiss_index.as_retriever(search_kwargs={"k": 3})

### 3. LLM 세팅 (GPT)

In [9]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

### 4. RetrievalQA 체인 구성

In [10]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)

### 5. 내 문서 기반 Q\&A 테스트

In [11]:
queries = [
    "국회의원은 몇 년마다 선출되나요?",
    "대한민국의 주권은 누구에게 있나요?",
    "헌법 제1조는 어떤 내용을 담고 있나요?"
]

for q in queries:
    print(f"\n[Q] {q}")
    print("[A]", qa_chain.run(q))


[Q] 국회의원은 몇 년마다 선출되나요?
[A] 국회의원은 대한민국 헌법에 따라 매 4년마다 선출됩니다.

[Q] 대한민국의 주권은 누구에게 있나요?
[A] 대한민국의 주권은 국민에게 있습니다.

[Q] 헌법 제1조는 어떤 내용을 담고 있나요?
[A] 헌법 제1조는 "이 헌법은 1988년 2월 25일부터 시행한다. 다만, 이 헌법을 시행하기 위하여 필요한 법률의 제정ㆍ개정과 이 헌법에 의한 대통령 및 국회의원의 선거 기타 이 헌법시행에 관한 준비는 이 헌법시행 전에 할 수 있다." 라는 내용을 담고 있습니다.


### 6. 사용자 입력 기반 챗봇 인터페이스 (기초형)
- 이후 Streamlit을 활용하면 GUI 웹 챗봇으로도 확장 가능합니다.

In [12]:
while True:
    query = input("\n질문을 입력하세요 (종료하려면 q): ")
    if query.lower() == 'q':
        break
    print("[답변]", qa_chain.run(query))

[답변] 제1조 이 헌법은 1988년 2월 25일부터 시행한다. 다만, 이 헌법을 시행하기 위하여 필요한 법률의 제정ㆍ개정과 이 헌법에 의한 대통령 및 국회의원의 선거 기타 이 헌법시행에 관한 준비는 이 헌법시행 전에 할 수 있다.


### 7. 정리
* 이 챕터에서는 지금까지 만든 인덱스를 활용하여,
* 실제로 "내 문서 기반 RAG 챗봇"을 구성해 보았습니다.
* 질문 → 검색 → 생성의 흐름이 하나의 체인으로 완성됩니다.

* 지금 챗봇은 동작은 하지만…

  * 검색 품질을 높일 순 없을까?
  * 응답 속도를 더 빠르게 할 수는 없을까?

> 다음 강의에서는 이 챗봇을 **개선하고 최적화**하는 고급 기법들 (Chunking, Hybrid Search 등)을 학습합니다.

